## Spatial Coverage Study

In [ ]:
# We create a new table to analyse the spatial coverage
df_sp = clean_nom_francais.copy()

print(df_sp.head())

# Make sure date/year exist
df_sp["date"]  = pd.to_datetime(df_sp["date"], errors="coerce")
df_sp = df_sp.dropna(subset=["date"])
df_sp["year"] = df_sp["date"].dt.year

# We ensure all columns contain only numerical values
df_sp["point_n"] = pd.to_numeric(df_sp["N° point"], errors="coerce")
df_sp["N° passage"] = pd.to_numeric(df_sp["N° passage"], errors="coerce")
df_sp["TOT_AV_sV"] = pd.to_numeric(df_sp["TOT_AV_sV"], errors="coerce")

# We group the relevant indicators by year
yearly = (
    df_sp.groupby("year", dropna=True)
      .agg(transects=("Nom transect", "nunique"),
           observers=("Nom observateur", "nunique"),
           departments=("code département", "nunique"),
           point_visits=("Nom transect", "size"),
           richness=("ESPECE", "nunique"),
           total_counts=("TOT_AV_sV", "sum"))
      .reset_index()
      .sort_values("year")
)
yearly["birds_per_visit"] = yearly["total_counts"] / yearly["point_visits"]

# We'll use this to calculate the coverage ratio per year
# To do that we take divide the unique (transect, point, pass) by the maximum number of (transects * points * passes) there can be (= transects * 10 * 2)
df_sp["combo"] = list(zip(df_sp["Nom transect"], df_sp["point_n"], df_sp["N° passage"]))

cov_y = (df_sp.dropna(subset=["year"])
           .groupby("year")
           .agg(obs_combos=("combo", "nunique"),
                max_passes=("N° passage", "max")))
cov_y = cov_y.join(yearly.set_index("year")[["transects"]])
cov_y["max_combos"] = cov_y["transects"] * 10 * 2 
cov_y["combo_coverage"] = cov_y["obs_combos"] / cov_y["max_combos"]
yearly = yearly.merge(cov_y[["combo_coverage"]], left_on="year", right_index=True, how="left")


# Coverage heatmap data 
Top_transects = 15 # Number of top transects by visits that will appear in the heatmap

ded = (df_sp.dropna(subset=["point_n", "N° passage"])
         .drop_duplicates(subset=["Nom transect", "year", "point_n", "N° passage"]))
cov = (ded.groupby(["Nom transect", "year"])
           .size()
           .unstack(fill_value=0))
if not cov.empty:
    keep = df_sp["Nom transect"].value_counts().head(Top_transects).index
    cov = cov.loc[cov.index.intersection(keep)]


# A) Transects sampled per year
plt.figure()
plt.plot(yearly["year"], yearly["transects"], marker="o")
plt.title("Transects sampled per year")
plt.xlabel("Year"); plt.ylabel("# transects")
plt.tight_layout(); plt.show()


# B) Coverage ratio per year
plt.figure()
plt.plot(yearly["year"], yearly["combo_coverage"], marker="o")
plt.title("Point–pass coverage ratio per year")
plt.xlabel("Year"); plt.ylabel("Observed / Max combos")
plt.tight_layout(); plt.show()


# C) Coverage heatmap (unique point–pass combos per transect-year)
plt.figure(figsize=(12, max(5, 0.3 * len(cov))))
im = plt.imshow(cov.values, aspect="auto")
plt.colorbar(im, fraction=0.02, pad=0.02, label="# point–pass combos")
plt.yticks(range(len(cov.index)), cov.index)
plt.xticks(range(len(cov.columns)), cov.columns, rotation=45)
plt.title("Coverage heatmap — unique (point, pass) combos per transect-year")
plt.xlabel("Year"); plt.ylabel("Transect")
plt.tight_layout(); plt.show()

Let's analyse these three figures, that represent different aspects of the spatial coverage.

**1) Transects sampled per year** 

Early years show fewer transects visited (around 41 in 2014), then a ramp-up through 2018 (around 65) and a rather stable plateau afterward (mostly between 62–65). So spatial coverage by number of transects expanded fast and then stabilized.


**2) Point–pass coverage ratio per year** 
Although the line looks dramatic at first, the y-axis is very tight: from around 0.91 to 1.00. The only clear drop is between 2019 and 2020, and after that, the coverage rebounds to around 0.99 and remains high. These graph shows there is a near-complete coverage most years, with 2020 as the outlier.

**3) Coverage heatmap (unique point×pass combos by transect–year, top-15 transects)**
On the first few years, we can see some gaps (`Borelie`, `Boucle du Vauclin`) and then some isolated holes  (`Hôtel des Plaisirs` around 2018–2019). Besides those, cells are almost always at the maximum (around 20 combos), which indicates a full pointxpass combo after 2016.

**Overall**
With the three figures, we can observe that there is a big increase in spatial coverage in the first years, followed by a stable coverage from 2018 onward. This sampling indicator looks strong and steady after the initial ramp-up, so besided the 2020 dip and a few early gaps, there isn't much added value in further analyzing this indicator. Because of that, we are going to dissect other ones. 



___
## Density Study

In [ ]:
df = clean_nom_francais.copy()
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["year"] = df["date"].dt.year

# --- Compute total birds per transect per year ---
transect_year_counts = (
    df.groupby(["year", "Nom transect"])["TOT_AV_V"]
      .sum()
      .reset_index()
      .rename(columns={"TOT_AV_V": "Bird_count"})
)

# --- Normalize counts per year ---
transect_year_counts["Normalized_density"] = transect_year_counts.groupby("year")["Bird_count"].transform(
    lambda x: x / x.max()
)
# --- color ---
transects = sorted(transect_year_counts["Nom transect"].unique())
colors = plt.cm.tab20(np.linspace(0, 1, len(transects)))  # up to 20 distinct colors
color_map = dict(zip(transects, colors))

# --- Plot normalized density distributions per year ---
years = sorted(transect_year_counts["year"].dropna().unique())

for yr in years:
    subset = transect_year_counts[transect_year_counts["year"] == yr]
    plt.figure(figsize=(10, 5))
    for _, row in subset.iterrows():
        plt.bar(
            row["Nom transect"], 
            row["Normalized_density"], 
            color=color_map[row["Nom transect"]],
            edgecolor="black",
            linewidth=0.5
        )
    plt.title(f"Normalized bird observation density per transect — {yr}")
    plt.xlabel("Transect")
    plt.ylabel("Normalized density (0–1)")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

In [ ]:
df["date"] = pd.to_datetime(df["date"], errors="coerce")
df["year"] = df["date"].dt.year

# --- Compute total birds per transect per year ---
transect_year_counts = (
    df.groupby(["year", "Nom transect"])["TOT_AV_V"]
      .sum()
      .reset_index()
      .rename(columns={"TOT_AV_V": "Bird_count"})
)

# --- Normalize globally (using the all-time maximum count) ---
global_max = transect_year_counts["Bird_count"].max()
transect_year_counts["Normalized_density"] = transect_year_counts["Bird_count"] / global_max

# --- Assign a consistent color per transect ---
transects = sorted(transect_year_counts["Nom transect"].unique())
colors = plt.cm.tab20(np.linspace(0, 1, len(transects)))  # up to 20 distinct colors
color_map = dict(zip(transects, colors))

# --- Plot evolution by year ---
years = sorted(transect_year_counts["year"].dropna().unique())

for yr in years:
    subset = transect_year_counts[transect_year_counts["year"] == yr].sort_values("Normalized_density", ascending=False)
    
    plt.figure(figsize=(10, 5))
    for _, row in subset.iterrows():
        plt.bar(
            row["Nom transect"], 
            row["Normalized_density"], 
            color=color_map[row["Nom transect"]],
            edgecolor="black",
            linewidth=0.5
        )
    
    plt.title(f"Bird density per transect — normalized (global max=1) — {yr}")
    plt.xlabel("Transect")
    plt.ylabel("Normalized density (0–1)")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()

## Bootstrap Method

In [ ]:
# --- Parameters ---
B = 1000  # number of bootstrap samples
alpha = 0.05  # for 95% CI

# --- Helper: bootstrap function ---
def bootstrap_ci(data, func=np.mean, B=1000, alpha=0.05):
    """
    Compute bootstrap confidence interval for a given statistic (func).
    """
    n = len(data)
    if n == 0:
        return np.nan, np.nan, np.nan  # handle empty groups
    
    # Original estimate
    theta_hat = func(data)
    
    # Bootstrap resamples
    boot_estimates = []
    for _ in range(B):
        sample = np.random.choice(data, size=n, replace=True)
        boot_estimates.append(func(sample))
    boot_estimates = np.array(boot_estimates)
    
    # Quantiles of bootstrap distribution
    q_low = np.quantile(boot_estimates, alpha / 2)
    q_high = np.quantile(boot_estimates, 1 - alpha / 2)
    
    # Reflected confidence interval (percentile method)
    ci_low = 2 * theta_hat - q_high
    ci_high = 2 * theta_hat - q_low
    
    return theta_hat, ci_low, ci_high



In [ ]:
# --- Compute bootstrap estimates per transect-year ---
bootstrap_results = []
for (year, transect), group in df.groupby(["year", "Nom transect"]):
    theta_hat, ci_low, ci_high = bootstrap_ci(group["TOT_AV_V"].values, func=np.sum, B=1000, alpha=0.05)
    bootstrap_results.append({
        "year": year,
        "transect": transect,
        "total_birds": theta_hat,
        "ci_low": ci_low,
        "ci_high": ci_high
    })

bootstrap_df = pd.DataFrame(bootstrap_results)

# --- Normalize by global maximum total (for comparable densities) ---
global_max = bootstrap_df["total_birds"].max()
bootstrap_df["density_norm"] = bootstrap_df["total_birds"] / global_max
bootstrap_df["ci_low_norm"] = bootstrap_df["ci_low"] / global_max
bootstrap_df["ci_high_norm"] = bootstrap_df["ci_high"] / global_max

display(bootstrap_df.head())

In [ ]:
years = sorted(bootstrap_df["year"].unique())

for yr in years:
    subset = bootstrap_df[bootstrap_df["year"] == yr].sort_values("density_norm", ascending=False)
    
    plt.figure(figsize=(10, 5))
    plt.bar(
        subset["transect"],
        subset["density_norm"],
        yerr=[subset["density_norm"] - subset["ci_low_norm"], subset["ci_high_norm"] - subset["density_norm"]],
        capsize=3
    )
    plt.title(f"Bootstrap-estimated normalized bird density per transect — {yr}")
    plt.xlabel("Transect")
    plt.ylabel("Normalized density (0–1)")
    plt.xticks(rotation=90)
    plt.tight_layout()
    plt.show()


In [ ]:
# --- Identify top 5 transects by mean normalized density across all years ---
top_transects = (
    bootstrap_df.groupby("transect")["density_norm"]
    .mean()
    .sort_values(ascending=False)
    .head(5)
    .index
)

print("Top 5 transects with highest mean normalized density:")
print(top_transects.tolist())

# --- Filter data for only those transects ---
top_df = bootstrap_df[bootstrap_df["transect"].isin(top_transects)]

# --- Plot temporal evolution for top 5 transects ---
plt.figure(figsize=(10, 6))
for transect, group in top_df.groupby("transect"):
    plt.plot(group["year"], group["density_norm"], marker="o", label=transect)

plt.title("Temporal evolution of normalized bird density — Top 5 transects (after bootstrapping)")
plt.xlabel("Year")
plt.ylabel("Normalized density (0–1)")
plt.legend(title="Transect", bbox_to_anchor=(1.05, 1), loc="upper left")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

### Shannon and simpson diversity index

In [ ]:
# --- Step 1: Aggregate counts per species per transect per year ---
species_counts = (
    df.groupby(["year", "Nom transect", "ESPECE"])["TOT_AV_V"]
    .sum()
    .reset_index()
    .rename(columns={"TOT_AV_V": "count"})
)

# --- Step 2: Define diversity index functions ---
def shannon_index(values):
    values = np.array(values)
    values = values[values > 0]
    proportions = values / values.sum()
    return -np.sum(proportions * np.log(proportions + 1e-12))

def simpson_index(values):
    values = np.array(values)
    values = values[values > 0]
    proportions = values / values.sum()
    return 1 - np.sum(proportions ** 2)


# ===========================
# 3️⃣ After bootstrapping — using bootstrap_df
# ===========================
boot_diversity = []
for year, group in bootstrap_df.groupby("year"):
    vals = group["density_norm"].values
    sh_mean, sh_low, sh_high = bootstrap_ci(vals, shannon_index, B=1000)
    si_mean, si_low, si_high = bootstrap_ci(vals, simpson_index, B=1000)
    boot_diversity.append({
        "year": year,
        "Shannon_boot": sh_mean,
        "Shannon_low": sh_low,
        "Shannon_high": sh_high,
        "Simpson_boot": si_mean,
        "Simpson_low": si_low,
        "Simpson_high": si_high
    })
boot_diversity = pd.DataFrame(boot_diversity)

# --- Plot Shannon index comparison ---
plt.figure(figsize=(9,5))
plt.plot(boot_diversity["year"], boot_diversity["Shannon_boot"], "o-", label="Bootstrap Shannon (mean)")
plt.fill_between(boot_diversity["year"], boot_diversity["Shannon_low"], boot_diversity["Shannon_high"], alpha=0.3, label="95% CI")
plt.title("Shannon diversity of transect densities — raw vs bootstrap (with CI)")
plt.xlabel("Year")
plt.ylabel("Shannon index (spatial diversity)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

# --- Plot Simpson index comparison ---
plt.figure(figsize=(9,5))
plt.plot(boot_diversity["year"], boot_diversity["Simpson_boot"], "s-", label="Bootstrap Simpson (mean)")
plt.fill_between(boot_diversity["year"], boot_diversity["Simpson_low"], boot_diversity["Simpson_high"], alpha=0.3, label="95% CI")
plt.title("Simpson diversity of transect densities — raw vs bootstrap (with CI)")
plt.xlabel("Year")
plt.ylabel("Simpson index (spatial diversity)")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()



## Linear regression of density + P-value

In [ ]:
# Compute mean density per year
mean_density = (
    transect_year_counts.groupby("year")["Normalized_density"]
    .mean()
    .reset_index()
    .sort_values("year")
)

# Define variables for regression
X = sm.add_constant(mean_density["year"])  # add intercept
y = mean_density["Normalized_density"]

# Fit the OLS (Ordinary Least Squares) regression model
model = sm.OLS(y, X).fit()

# Print model summary
print(model.summary())

# Extract the p-value for the year coefficient
p_value = model.pvalues["year"]
slope = model.params["year"]
r_squared = model.rsquared

print(f"Slope: {slope:.6f}")
print(f"p-value (year effect): {p_value:.6f}")
print(f"R²: {r_squared:.4f}")

# Predict fitted values and confidence interval
predictions = model.get_prediction(X)
pred_summary = predictions.summary_frame(alpha=0.05)

# Plot data + regression line + 95% confidence interval
plt.figure(figsize=(9, 6))
plt.scatter(mean_density["year"], mean_density["Normalized_density"], label="Observed mean densities")
plt.plot(mean_density["year"], pred_summary["mean"], label="Fitted regression line", linewidth=2)
plt.fill_between(
    mean_density["year"],
    pred_summary["mean_ci_lower"],
    pred_summary["mean_ci_upper"],
    alpha=0.3,
    label="95% confidence interval"
)
plt.title("Linear regression of mean normalized density over years")
plt.xlabel("Year")
plt.ylabel("Mean normalized density")
plt.legend()
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()

# Summary of Density indicator (what we did)

## 🪶 Density Indicator — Summary of Computations

### 1. Definition
The density indicator measures the relative abundance of birds observed per transect and year.



### 2. Computation Steps

**(a) Counting per Transect and Year**  
From the cleaned observation dataset (`nom_francais_clean`), total bird counts were aggregated by `(year, transect)` using columns such as `TOT_A`, `TOT_V_sV`, etc.

**(b) Normalization**  
Each transect’s annual count was normalized by the maximum count observed across all years:

$$
\text{density\_norm}_{i,t} = \frac{\text{count}_{i,t}}{\max(\text{count}_{\text{all years}})}
$$

Densities are thus scaled to the range [0, 1].



### 3. Bootstrap Estimation

A bootstrap resampling method was used to estimate uncertainty:

1. For each year, resample transects with replacement \( B \) times (e.g. \( B = 1000 \)).
2. Compute the mean normalized density for each resample.
3. Obtain 95% confidence intervals from the empirical quantiles of the bootstrap distribution.

$$
\text{CI}_{95\%} = [\hat{\theta}^*_{2.5\%}, \hat{\theta}^*_{97.5\%}]
$$



### 4. Derived Indicators and Visualization

- Annual mean normalized density computed and plotted over time.  
- Per-year density distribution visualized by transect (color-coded).  
- Temporal evolution of normalized densities visualized across transects.  
- Bootstrap mean and confidence intervals plotted for density trends.


### 5. Diversity Indices Applied to Density

**Shannon Diversity Index**
$$
H' = -\sum_i p_i \ln(p_i)
$$

**Simpson Diversity Index**
$$
D = 1 - \sum_i p_i^2
$$

Both indices computed from the normalized density distribution across transects:
- Calculated per year on raw and bootstrapped densities.
- Confidence intervals estimated via bootstrap resampling.
- Temporal evolution of both indices visualized.
- Additional plots generated for the top 5 transects with highest mean density.


### 6. Linear Regression and p-Value Computation

A **linear regression** was fitted using `statsmodels` to assess the presence of a temporal trend in mean normalized density:

$$
\text{density\_norm} = \beta_0 + \beta_1 \times \text{year} + \varepsilon
$$

- The regression was estimated with the **Ordinary Least Squares (OLS)** method.  
- The **slope** (\(\beta_1\)) quantifies the direction and magnitude of the density trend over years.  
- The **p-value** for the `year` coefficient tests the null hypothesis \(H_0: \beta_1 = 0\).  
- A **95% confidence interval** and **R²** statistic were also extracted to evaluate model fit.  
- The fitted regression line and its 95% confidence band were plotted together with observed mean densities.





___
## Detectability Study (auditory vs visual)

In [ ]:
TOP_N = 5  # number of transects we analyze (ordered by most visited)


df_det = clean_nom_francais.copy()
df_det["date"] = pd.to_datetime(df_det["date"], errors="coerce")
df_det["year"] = pd.to_numeric(df_det["date"].dt.year, errors="coerce")
df_det["N° passage"] = pd.to_numeric(df_det["N° passage"], errors="coerce")


df_det = df_det.dropna(subset=["Nom transect", "year"])


ranking = df_det.groupby("Nom transect").size().sort_values(ascending=False)
top_transects = ranking.head(TOP_N).index.tolist() 
df_det_top = df_det[df_det["Nom transect"].isin(top_transects)]

# -------- AGGREGATE & SHARES --------
agg = (df_det_top.groupby(["Nom transect", "year"], dropna=False)
              .agg(A_sum=("TOT_A","sum"), V_sum=("TOT_V_sV","sum"))
              .reset_index())
total = (agg["A_sum"] + agg["V_sum"])
agg["Auditory_pct"] = 100 * agg["A_sum"] / total
agg["Visual_pct"]   = 100 - agg["Auditory_pct"]

# Plot the histogram
n = len(top_transects)
ncols = min(3, n)
nrows = int(np.ceil(n / ncols))
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 10), squeeze=False)

for i, tr in enumerate(top_transects):
    ax = axes[i // ncols, i % ncols]
    sub = agg[agg["Nom transect"] == tr].sort_values("year")
    years = sub["year"].astype(int).tolist()
    a = sub["Auditory_pct"].values
    v = sub["Visual_pct"].values

    ax.bar(years, a, label="Auditory (%)")
    ax.bar(years, v, bottom=a, label="Visual (%)")
    ax.set_title(f"{tr} — Modality mix")
    ax.set_xlabel("Year"); ax.set_ylabel("Share (%)")
    ax.set_ylim(0, 100)
    ax.set_xticks(years); ax.set_xticklabels(years, rotation=45)

# Hide empty axes
for j in range(i+1, nrows*ncols):
    axes[j // ncols, j % ncols].axis("off")

handles, labels = axes[0,0].get_legend_handles_labels()
fig.legend(handles, labels, loc="upper center", ncol=2, bbox_to_anchor=(0.5, 1.03))
fig.tight_layout()
plt.show()


Observing the graphs for each of the top five most visited transects, we can see that for some of them `(Morne Babet, Habitation Petit Riviere, Moulin a vent)`, there seems to be an increase in the share of auditory observations of bird. 

In order to test our hypothesis if there are trends in some of these transects, we are going to present an initial hypothesis $H_0$: $\beta_1 = 0$, with $\beta_1$ being the slope on each year 

In [ ]:
from statsmodels.stats.proportion import proportion_confint

agg["AV_total"] = (agg["A_sum"] + agg["V_sum"]).astype(float)



ci_low, ci_high = proportion_confint(
    count=agg["A_sum"].astype(float),
    nobs=agg["AV_total"].astype(float),
    alpha=0.05,
    method="wilson"
)
agg.loc["Auditory_pct_low"]  = 100 * ci_low
agg.loc["Auditory_pct_high"] = 100 * ci_high

# We fit a binomial GLM for the % auditory ~ year (per transect)
def fit_binom_glm(sub_df):
    """sub_df has columns: year, A_sum, V_sum, AV_total (>0 rows). Returns pred df + slope, pval."""
    df = sub_df.sort_values("year").copy()

    df["year_c"] = df["year"] - df["year"].mean()  
    y = (df["A_sum"] / df["AV_total"]).values
    X = sm.add_constant(df[["year_c"]])
    model = sm.GLM(y, X, family=sm.families.Binomial(), var_weights=df["AV_total"].values)
    res = model.fit(cov_type="HC1")

    pred = res.get_prediction(X)
    ci = pred.conf_int(alpha=0.05)
    df["pred"]   = pred.predicted_mean
    df["ci_low"] = ci[:, 0]
    df["ci_high"]= ci[:, 1]

    slope = float(res.params.get("year_c"))  
    pval  = float(res.pvalues.get("year_c"))
    return df, slope, pval

# We plot the observed auditory (%) bars with Wilson CIs and GLM fit
transects = top_transects
n = len(transects)
ncols = min(3, n)
nrows = int(np.ceil(n / ncols)) if n else 1
fig, axes = plt.subplots(nrows=nrows, ncols=ncols, figsize=(16, 10), squeeze=False)
axes = axes.flatten() if nrows*ncols > 1 else [axes]

for i, tr in enumerate(transects):
    ax = axes[i]
    sub = agg[agg["Nom transect"] == tr].sort_values("year").copy()
    years = sub["year"].astype(int).to_numpy()
    y_bar = sub["Auditory_pct"].to_numpy()

    # Bars: observed auditory %
    ax.bar(years, y_bar, label="Observed Auditory (%)", alpha=0.85)

    # Wilson error bars (only where defined)
    y_low  = sub["Auditory_pct_low"].to_numpy()
    y_high = sub["Auditory_pct_high"].to_numpy()
    yerr = np.vstack([y_bar - y_low, y_high - y_bar])
    ax.errorbar(years, y_bar, yerr=yerr, fmt="none", capsize=3, linewidth=1, ecolor="black")

    # GLM fit line + 95% CI band
    fit_df, slope, pval = fit_binom_glm(sub)
    ax.plot(fit_df["year"].astype(int), fit_df["pred"]*100, marker="o",
                linestyle="-", linewidth=1.5, label="Binomial fit")
    ax.fill_between(fit_df["year"].astype(int),
                    fit_df["ci_low"]*100, fit_df["ci_high"]*100,
                    alpha=0.20, label="95% CI (fit)")

    ax.set_title(f"{tr} — Observed Auditory (%)\nlogit-slope={slope:.3f}, p={pval:.3g}")
    ax.set_xlabel("Year"); ax.set_ylabel("Auditory (%)")
    ax.set_ylim(0, 100)
    ax.set_xticks(years); ax.set_xticklabels(years, rotation=45)

# Hide unused subplots 
for j in range(i + 1, nrows * ncols):
    axes[j].axis("off")

handles, labels = axes[0].get_legend_handles_labels()
if handles:
    fig.legend(handles, labels, loc="upper center", ncol=3, bbox_to_anchor=(0.5, 1.02))

fig.tight_layout()
plt.show()

# Conclusion

After tracing these figures, with their corresponding CIs, p_values and slopes, we can see that for: 
1. Morne Babet: The p_value is much smaller than 0.05. We can say that the trend is highly significant, and that there is a strong and steady increase in the proportion of auditory detections over time.
2. Habitation Petit Rivière: Same as Morne Babet
3. Moulin à Vent: The p_value is smaller than 0.05. We can say that the trend is signicant, and that there is more variability across the years, but the fitted line still shows a positive slope.
4. Là-Haut: The p_value is much bigger than 0.05. We can say that the trend is not significant, and that the modality mix remains roughly constant over time.
5. Hôtel des Plaisirs: the p_value is bigger than 0.05. We can say that the trend is not significant, and although the slope is slightly upward, the uncertainty is large, so there isn't any statistically significant trend in auditory share.


# Summary of Modality-Mix Indicator (Auditory vs Visual)

## 1. Definition
The modality-mix indicator measures, for each transect and year, the share of detections that were auditory and visual:

$$
\frac{A_{y,t}}{A_{y,t} + V_{y,t}}
$$

where $A_{y,t}=\text{auditory detections}$ and $V_{y, t} = \text{visual detections}$ for each year $y$ at transect $t$.

## 2. Computation Steps

**(a) Counting per Transect and Year**

From the cleaned observation dataset `(nom_francais_clean)`, total audio/visual bird counts were aggregated by `(year, transect)` adding columns `TOT_A` and `TOT_V_sV`.

**(b) Observed shares**

To calculate the percentage of observed audio shares, we do:
$$
\widehat{p}_{y,t}=\frac{TOT\_A}{AV\_total},\qquad \text{Auditory\%}=100\times \widehat{p}_{y,t}
$$

## 3. Confidence Intervals for Observed Shares (Wilson method)

We use the Wilson method, because classic "Wald" Confidence Intervalls perform poorly with small $n$ or extreme $p$ (bounds can leave [0, 1]) 
Instea, Wilson score intervals have better coverage and stay in [0, 1], which matters because if we increase the number of transects, the $\text{year} \times \text{transect}$ might have modest totals.

Formula for $95\%$ CI:

Let $z = 1.96$, $n = \text{AV\_total}$, $\widehat{p} = \frac{\text{TOT\_A}}{n}$. Then

$$
\text{CI}_{\text{Wilson}} = \frac{\widehat{p} + \frac{z^2}{2n} \pm z\sqrt{\frac{\widehat{p}(1 - \widehat{p})}{n} + \frac{z^2}{4n^2}}}{1 + \frac{z^2}{n}}
$$

In the code, this is calculated by the imported function `statsmodels.stats.proportion.proportion_confint(method="wilson")`, then we multiply it by 100 to get the percentage.

## 4. Binomial Generalized Linear Model

We use a GLM because the share of auditory detections is a proportion, each value comes from a count of auditory detections out of a total number of detections, so it follows a binomial distribution. Also, a GLM with a binomial family and a logit link models how the probability of an auditory detection changes with time (year), while taking into account that the years with more detections provide more reliable estimates. 

Per each transect, we utilise the following model:

$$
\text{logit}(\text{Pr}[\text{auditory}|y,t]) = \beta_0 + \beta_1 (y - \widehat{y})
$$

With:
1. $\beta_0$: intercept
2. $\beta_1$: slope on year
3. $(y-\hat y)$: centered year

To calculate the p_value $p$ of each transect, we are going to do the following computations:

$$
z = \frac{\hat \beta_1}{\text{SE}(\hat \beta_1)d},\qquad p=2(1 - \phi(|z|))
$$

Where $\phi(\cdot)$ is the standard normal CDF.

# Part 3

In [ ]:
# Compute total abundance per species
species_counts = (
    df.groupby("ESPECE")["TOT_AV_V"]
    .sum()
    .sort_values(ascending=False)
)

top5_species = species_counts.head(5).index.tolist()
print("Top 5 most observed species:", top5_species)


In [ ]:
top5_data = (
    df[df["ESPECE"].isin(top5_species)]
    .groupby(["year", "ESPECE"])["TOT_AV_V"]
    .sum()
    .reset_index()
)

In [ ]:
plt.figure(figsize=(10, 6))

for species in top5_species:
    sub = top5_data[top5_data["ESPECE"] == species]
    plt.plot(sub["year"], sub["TOT_AV_V"], marker="o", label=species)

plt.title("Temporal Evolution of Abundance for Top 5 Most Observed Bird Species")
plt.xlabel("Year")
plt.ylabel("Total Recorded Abundance (TOT_AV_V)")
plt.legend(title="Species", bbox_to_anchor=(1.05,1), loc="upper left")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()


### Normalization to compare trends

In [ ]:
top5_data_norm = top5_data.copy()
top5_data_norm["scaled"] = top5_data_norm.groupby("ESPECE")["V_V"].transform(
    lambda s: (s - s.min()) / (s.max() - s.min() + 1e-9)
)

plt.figure(figsize=(10, 6))
for species in top5_species:
    sub = top5_data_norm[top5_data_norm["ESPECE"] == species]
    plt.plot(sub["year"], sub["scaled"], marker="o", label=species)

plt.title("Normalized Temporal Trends of Top 5 Bird Species (0–1 scale)")
plt.xlabel("Year")
plt.ylabel("Relative abundance (normalized)")
plt.legend(title="Species", bbox_to_anchor=(1.05,1), loc="upper left")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()


### Bootstrap + CIs

In [ ]:
bootstrap_results = []

for species in top5_species:
    sp = df[df["ESPECE"] == species]
    
    for year, group in sp.groupby("year"):
        counts = group["TOT_AV_V"].values
        
        mean, ci_low, ci_high = bootstrap_ci(counts,func=np.mean, B=1000, alpha=0.05)
        
        bootstrap_results.append({
            "species": species,
            "year": year,
            "mean_abundance": mean,
            "ci_low": ci_low,
            "ci_high": ci_high
        })

bootstrap_df_species = pd.DataFrame(bootstrap_results).sort_values(["species","year"])
bootstrap_df_species.head()


In [ ]:
plt.figure(figsize=(10, 6))

for species, group in bootstrap_df_species.groupby("species"):
    plt.plot(group["year"], group["mean_abundance"], marker="o", label=species)
    plt.fill_between(group["year"], group["ci_low"], group["ci_high"], alpha=0.2)

plt.title("Temporal Evolution of Bird Abundance (Top 5 Species) with Bootstrap 95% CI")
plt.xlabel("Year")
plt.ylabel("Mean Recorded Abundance (TOT_AV_V)")
plt.legend(title="Species", bbox_to_anchor=(1.05,1), loc="upper left")
plt.grid(True, linestyle="--", alpha=0.6)
plt.tight_layout()
plt.show()


In [ ]:
for species, group in bootstrap_df_species.groupby("species"):
    plt.figure(figsize=(8,5))
    plt.plot(group["year"], group["mean_abundance"], marker="o")
    plt.fill_between(group["year"], group["ci_low"], group["ci_high"], alpha=0.3)
    plt.title(f"Annual Abundance of {species} (with 95% Bootstrap CI)")
    plt.xlabel("Year")
    plt.ylabel("Mean Recorded Abundance (TOT_AV_V)")
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.tight_layout()
    plt.show()


## Trend Significance per Species

In [ ]:
trend_results = []

for species, group in bootstrap_df_species.groupby("species"):
    
    # Sort by year to ensure correct alignment
    group = group.sort_values("year")
    
    # Linear regression test
    X = sm.add_constant(group["year"])   # predictor: year
    y = group["mean_abundance"]         # response: bootstrapped mean abundance
    model = sm.OLS(y, X).fit()
    
    slope = model.params["year"]
    p_slope = model.pvalues["year"]
    r2 = model.rsquared
    
    # Kendall Tau Non-parametric trend test
    tau, p_kendall = kendalltau(group["year"], group["mean_abundance"])
    
    trend_results.append({
        "species": species,
        "slope (OLS)": slope,
        "p-value (OLS slope)": p_slope,
        "R² (trend fit)": r2,
        "Kendall Tau": tau,
        "p-value (Kendall)": p_kendall
    })

trend_results_df = pd.DataFrame(trend_results)
trend_results_df

For all this species except "Quiscale merle" , the slope is > 0 , so species abundance is increasing over time, but the only significant values are for "Elénie siffleuse" and 	"Tourterelle à queue carrée". "Quiscale merle" has also a significant negative value, so could suppose that the abundance is decreasing over the time, but the p-value is quite high. This suggests a possible decline, but evidence is weak. By looking at p-value (< 0.05), we can say that trend is statistically significant for "Elénie siffleuse" and "Tourterelle à queue carrée". Looking at Kendall p-value (< 0.05) trend is significant even without assuming linearity for these two species.

### Trend Line + Bootstrap CI + Observed Points (per species)

In [ ]:
for species, group in bootstrap_df_species.groupby("species"):
    
    group = group.sort_values("year")  # ensure correct order
    years = group["year"].values
    y = group["mean_abundance"].values
    
    # Fit regression for the species
    X = sm.add_constant(years)
    model = sm.OLS(y, X).fit()
    
    # Predictions + Confidence Interval for trend line
    pred = model.get_prediction(X).summary_frame(alpha=0.05)
    y_fit = pred["mean"]
    y_low = pred["mean_ci_lower"]
    y_high = pred["mean_ci_upper"]
    
    # Plot
    plt.figure(figsize=(8,5))
    
    # Bootstrap CI (vertical uncertainty)
    plt.fill_between(years, group["ci_low"], group["ci_high"], alpha=0.25, label="Bootstrap 95% CI (Abundance)")
    
    # Observed points
    plt.plot(years, y, marker="o", linewidth=1.5, label="Mean Recorded Abundance")
    
    # Linear regression trend
    plt.plot(years, y_fit, linewidth=2.2, label="Fitted Trend (OLS)")
    
    # Confidence band for trend
    plt.fill_between(years, y_low, y_high, alpha=0.2, label="95% CI (Trend)")
    
    plt.title(f"Trend in Annual Abundance — {species}")
    plt.xlabel("Year")
    plt.ylabel("Mean Abundance (TOT_AV_V)")
    plt.grid(True, linestyle="--", alpha=0.6)
    plt.legend(loc="best")
    plt.tight_layout()
    plt.show()


### Interpretation of Species-Specific Abundance Trends

The figures above display the temporal evolution of abundance for the five most frequently recorded bird species in the dataset, using the standardized mean abundance (TOT_AV_V) per year and associated uncertainty estimated via bootstrap resampling. For **Élenie siffleuse**, the annual mean abundance shows a generally increasing pattern over the study period, and the fitted linear trend is positive with confidence intervals that do not overlap heavily with zero, indicating a statistically supported rise in abundance. A similar positive and statistically significant trend is observed for **Tourterelle à queue carrée**, where both the slope and the bootstrap confidence intervals suggest a sustained increase in occurrence intensity over time. 

In contrast, **Quiscale merle** shows a declining fitted trend line, but the year-to-year variability is relatively high and the confidence intervals are broader, resulting in a non-significant trend. This suggests that although the species may be experiencing a reduction in recorded abundance, further data or more controlled sampling would be needed to confidently confirm this decline. For **Sporophile rougegorge** and **Sucrier à ventre jaune**, the mean abundance values fluctuate from year to year without displaying a marked directional change. Their fitted trends are near-flat and associated confidence intervals are wide, indicating that these populations have remained relatively stable across the studied period.

Overall, these results highlight **two species experiencing significant increases** (Élenie siffleuse and Tourterelle à queue carrée), **one species with a possible but unconfirmed decline** (Quiscale merle), and **two species showing stable abundance levels with no evidence of directional long-term change** (Sporophile rougegorge and Sucrier à ventre jaune).


## Limitations

The mean-abundance and bootstrap approach provides a robust, effort-standardized indicator of species presence over time. However, this method does not explicitly correct for variation in detection probability (observer effects, weather, time of day), and assumes independence between sampling events, which may not always hold. Additionally, mean abundance reflects relative observation rates rather than absolute population sizes, because mean abundance is how often the species are recorded, not how many individuals exist in the ecosystem., and the linear trend model may not capture non-linear ecological dynamics. Therefore, while the approach reliably identifies broad directional changes, results should be interpreted as population indices rather than direct population estimates.